<h1><u><b>MULTI-LABEL CLASSIFICATION</b></u></h1>

The other approaches which could be used for other similar problem but is not suitable for this particular case are:
<ul>
    <li><b>Tensorflow_Ranking</b>: 
        This is the latest neaural network based ranking model for massive amount of data(which we pretty much dont have), training it on relatively smaller datasets can result in very poor results. This will give us a ranked order of the top n universities that the candidate should consider applying, but this is not justified as a candidate with very good score and academics may have greater number of options than a candidate with very less marks and academically weak, we can not provide same number of options for both the candidates.
        
   <li><b>Multi-Class Classification</b>:
        This will provide one output university that the candidate should consider applying to, this is not the right thing to do as th candidate does not apply to only one university and always likes more options, therfore this solution is nit accepted.
        
<li><b>Multi-Label Classification</b>:
    Therefore we have to adapt a more adaptive solution, which can display more results to a person who has person who good academics and relatively lesser and more feasible options to a person with weaker academic records. 

In [1]:
import collections
import pandas
from collections import defaultdict
import numpy as np 
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
from numpy.random import permutation
import math

In [2]:
data=pd.read_csv('data.csv')

In [3]:
data=data.drop(['Unnamed: 0'],1)

In [4]:
data

,userName,toeflScore,greV,greQ,greA,termAndYear,ugCollege,cgpa,univName
0,ninny.rgs,101.0,430.0,740.0,4.5,Fall - 2011,abhinavhitechcollegeofengineering,0.7227,University of Southern California
1,ninny.rgs,101.0,159.0,159.0,4.5,Fall - 2011,abviiitm,0.7227,New Jersey Institute of Technology
2,aashayshah,88.0,147.0,164.0,2.5,Spring - 2016,aceengineeringcollege,0.6200,University of Texas Arlington
3,cjaganmohan,102.0,140.0,148.0,3.0,Fall - 2013,acharyainstituteoftechnology,0.7400,University of Texas Arlington
4,pvskmz,115.0,154.0,163.0,3.0,Fall - 2013,acharyanagarjunauniversity,0.7100,Arizona State University
5,anu_661,105.0,154.0,163.0,3.0,Fall - 2011,acharyanagarjunauniversity,0.8900,SUNY Stony Brook
6,nachiappan123,100.0,157.0,163.0,3.0,Fall - 2015,acharyanagarjunauniversity,0.8004,Arizona State University
7,username123456,95.0,143.0,162.0,3.0,Spring - 2014,acharyanagarjunauniversity,0.8500,University of Texas Arlington
8,username123456,95.0,143.0,162.0,3.0,Spring - 2014,acharyanagarjunauniversity,0.8500,University of North Carolina Charlotte
9,sam_the_killer,89.0,370.0,690.0,4.0,Fall - 2007,acharyanagarjunauniversity,0.6941,University of Texas Arlington


In [5]:
one_hot = pd.get_dummies(data['univName'])

In [6]:
data = data.drop('univName',axis = 1)

In [7]:
data = data.join(one_hot)

In [8]:
data

,userName,toeflScore,greV,greQ,greA,termAndYear,ugCollege,cgpa,Arizona State University,Carnegie Mellon University,...,University of Pennsylvania,University of Southern California,University of Texas Arlington,University of Texas Austin,University of Texas Dallas,University of Utah,University of Washington,University of Wisconsin Madison,Virginia Polytechnic Institute and State University,Wayne State University
0,ninny.rgs,101.0,430.0,740.0,4.5,Fall - 2011,abhinavhitechcollegeofengineering,0.7227,0,0,...,0,1,0,0,0,0,0,0,0,0
1,ninny.rgs,101.0,159.0,159.0,4.5,Fall - 2011,abviiitm,0.7227,0,0,...,0,0,0,0,0,0,0,0,0,0
2,aashayshah,88.0,147.0,164.0,2.5,Spring - 2016,aceengineeringcollege,0.6200,0,0,...,0,0,1,0,0,0,0,0,0,0
3,cjaganmohan,102.0,140.0,148.0,3.0,Fall - 2013,acharyainstituteoftechnology,0.7400,0,0,...,0,0,1,0,0,0,0,0,0,0
4,pvskmz,115.0,154.0,163.0,3.0,Fall - 2013,acharyanagarjunauniversity,0.7100,1,0,...,0,0,0,0,0,0,0,0,0,0
5,anu_661,105.0,154.0,163.0,3.0,Fall - 2011,acharyanagarjunauniversity,0.8900,0,0,...,0,0,0,0,0,0,0,0,0,0
6,nachiappan123,100.0,157.0,163.0,3.0,Fall - 2015,acharyanagarjunauniversity,0.8004,1,0,...,0,0,0,0,0,0,0,0,0,0
7,username123456,95.0,143.0,162.0,3.0,Spring - 2014,acharyanagarjunauniversity,0.8500,0,0,...,0,0,1,0,0,0,0,0,0,0
8,username123456,95.0,143.0,162.0,3.0,Spring - 2014,acharyanagarjunauniversity,0.8500,0,0,...,0,0,0,0,0,0,0,0,0,0
9,sam_the_killer,89.0,370.0,690.0,4.0,Fall - 2007,acharyanagarjunauniversity,0.6941,0,0,...,0,0,1,0,0,0,0,0,0,0


In [9]:
temp=data.copy()

In [10]:
comp=data.copy()

In [11]:
temp

,userName,toeflScore,greV,greQ,greA,termAndYear,ugCollege,cgpa,Arizona State University,Carnegie Mellon University,...,University of Pennsylvania,University of Southern California,University of Texas Arlington,University of Texas Austin,University of Texas Dallas,University of Utah,University of Washington,University of Wisconsin Madison,Virginia Polytechnic Institute and State University,Wayne State University
0,ninny.rgs,101.0,430.0,740.0,4.5,Fall - 2011,abhinavhitechcollegeofengineering,0.7227,0,0,...,0,1,0,0,0,0,0,0,0,0
1,ninny.rgs,101.0,159.0,159.0,4.5,Fall - 2011,abviiitm,0.7227,0,0,...,0,0,0,0,0,0,0,0,0,0
2,aashayshah,88.0,147.0,164.0,2.5,Spring - 2016,aceengineeringcollege,0.6200,0,0,...,0,0,1,0,0,0,0,0,0,0
3,cjaganmohan,102.0,140.0,148.0,3.0,Fall - 2013,acharyainstituteoftechnology,0.7400,0,0,...,0,0,1,0,0,0,0,0,0,0
4,pvskmz,115.0,154.0,163.0,3.0,Fall - 2013,acharyanagarjunauniversity,0.7100,1,0,...,0,0,0,0,0,0,0,0,0,0
5,anu_661,105.0,154.0,163.0,3.0,Fall - 2011,acharyanagarjunauniversity,0.8900,0,0,...,0,0,0,0,0,0,0,0,0,0
6,nachiappan123,100.0,157.0,163.0,3.0,Fall - 2015,acharyanagarjunauniversity,0.8004,1,0,...,0,0,0,0,0,0,0,0,0,0
7,username123456,95.0,143.0,162.0,3.0,Spring - 2014,acharyanagarjunauniversity,0.8500,0,0,...,0,0,1,0,0,0,0,0,0,0
8,username123456,95.0,143.0,162.0,3.0,Spring - 2014,acharyanagarjunauniversity,0.8500,0,0,...,0,0,0,0,0,0,0,0,0,0
9,sam_the_killer,89.0,370.0,690.0,4.0,Fall - 2007,acharyanagarjunauniversity,0.6941,0,0,...,0,0,1,0,0,0,0,0,0,0


In [12]:
temp.groupby(['userName'])

In [13]:
comp=comp.groupby(['userName']).max()

In [14]:
comp

,toeflScore,greV,greQ,greA,termAndYear,ugCollege,cgpa,Arizona State University,Carnegie Mellon University,Clemson University,...,University of Pennsylvania,University of Southern California,University of Texas Arlington,University of Texas Austin,University of Texas Dallas,University of Utah,University of Washington,University of Wisconsin Madison,Virginia Polytechnic Institute and State University,Wayne State University
userName,,,,,,,,,,,,,,,,,,,,,
! Superman,118.0,165.0,169.0,5.5,Fall - 2014,People's Education Society Institute of Techno...,0.9100,0,0,0,...,1,0,0,1,0,0,0,0,0,0
!mpulse,95.0,140.0,166.0,3.0,Fall - 2013,VTU,0.8280,0,0,0,...,0,0,0,0,0,0,0,0,0,0
$4R4,109.0,145.0,168.0,3.0,Fall - 2015,NITK Surathkal,0.8970,0,0,0,...,0,1,0,0,0,0,0,0,0,0
$aumil,109.0,610.0,750.0,3.0,Fall - 2013,VJTI,0.8600,0,0,0,...,0,1,0,0,0,0,0,0,0,0
(:)_(:),103.0,150.0,166.0,3.0,Fall - 2013,NIT Rourkela,0.0764,0,0,0,...,0,0,0,0,0,1,0,0,0,0
++ZeuS++,110.0,660.0,760.0,4.0,Fall - 2011,University of Mumbai,0.6530,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-=[prat17]=-,85.0,152.0,161.0,4.5,Fall - 2012,nit,0.7600,0,0,1,...,0,0,0,0,0,0,0,0,0,0
-KaaL-,97.0,142.0,157.0,3.5,Fall - 2012,University of Mumbai,0.6437,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-MK-,99.0,151.0,159.0,3.5,Fall - 2015,sardarpatelcollegeofengineering,0.7580,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
len(comp.columns)

50

In [16]:
from sklearn import preprocessing
number = preprocessing.LabelEncoder()
comp['termAndYear'] = number.fit_transform(comp.termAndYear)
comp['ugCollege'] = number.fit_transform(comp.ugCollege)


In [17]:
comp

,toeflScore,greV,greQ,greA,termAndYear,ugCollege,cgpa,Arizona State University,Carnegie Mellon University,Clemson University,...,University of Pennsylvania,University of Southern California,University of Texas Arlington,University of Texas Austin,University of Texas Dallas,University of Utah,University of Washington,University of Wisconsin Madison,Virginia Polytechnic Institute and State University,Wayne State University
userName,,,,,,,,,,,,,,,,,,,,,
! Superman,118.0,165.0,169.0,5.5,20,156,0.9100,0,0,0,...,1,0,0,1,0,0,0,0,0,0
!mpulse,95.0,140.0,166.0,3.0,17,565,0.8280,0,0,0,...,0,0,0,0,0,0,0,0,0,0
$4R4,109.0,145.0,168.0,3.0,21,66,0.8970,0,0,0,...,0,1,0,0,0,0,0,0,0,0
$aumil,109.0,610.0,750.0,3.0,17,559,0.8600,0,0,0,...,0,1,0,0,0,0,0,0,0,0
(:)_(:),103.0,150.0,166.0,3.0,17,43,0.0764,0,0,0,...,0,0,0,0,0,1,0,0,0,0
++ZeuS++,110.0,660.0,760.0,4.0,14,524,0.6530,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-=[prat17]=-,85.0,152.0,161.0,4.5,16,831,0.7600,0,0,1,...,0,0,0,0,0,0,0,0,0,0
-KaaL-,97.0,142.0,157.0,3.5,16,524,0.6437,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-MK-,99.0,151.0,159.0,3.5,21,938,0.7580,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
x = comp.iloc[:,0:7] 
y = comp.iloc[:,7:50]

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.30, random_state = 0)

In [20]:
X_test

,toeflScore,greV,greQ,greA,termAndYear,ugCollege,cgpa
userName,,,,,,,
prasanna1433,93.0,144.0,161.0,3.0,20,387,0.7750
kirti8192,113.0,164.0,164.0,4.0,17,1016,0.9270
GT-R,112.0,157.0,155.0,4.0,16,737,0.7580
karthikraoa,114.0,640.0,800.0,3.5,14,66,0.8700
priyatham,99.0,460.0,800.0,3.5,14,839,0.8070
yuvarajchiyyodu,104.0,167.0,170.0,3.0,17,337,0.7200
tourniquet,111.0,162.0,161.0,4.5,17,202,0.7590
sahildude,106.0,164.0,158.0,3.0,17,913,0.7400
abhishekjoshi,117.0,164.0,170.0,5.5,21,1155,0.9570


In [21]:
import scipy.sparse as sp
X_train = sp.csr_matrix(X_train)
Y_train = sp.csr_matrix(Y_train)
X_test = sp.csr_matrix(X_test)
Y_test = sp.csr_matrix(Y_test)

In [22]:
sam_x = comp.iloc[:,0:7] 
sam_y = comp.iloc[:,7:50]

In [23]:
sam_x

,toeflScore,greV,greQ,greA,termAndYear,ugCollege,cgpa
userName,,,,,,,
! Superman,118.0,165.0,169.0,5.5,20,156,0.9100
!mpulse,95.0,140.0,166.0,3.0,17,565,0.8280
$4R4,109.0,145.0,168.0,3.0,21,66,0.8970
$aumil,109.0,610.0,750.0,3.0,17,559,0.8600
(:)_(:),103.0,150.0,166.0,3.0,17,43,0.0764
++ZeuS++,110.0,660.0,760.0,4.0,14,524,0.6530
-=[prat17]=-,85.0,152.0,161.0,4.5,16,831,0.7600
-KaaL-,97.0,142.0,157.0,3.5,16,524,0.6437
-MK-,99.0,151.0,159.0,3.5,21,938,0.7580


In [24]:
from sklearn.model_selection import train_test_split
samxtrain, samxtest, samytrain, samytest = train_test_split(sam_x, sam_y, test_size = 0.30, random_state = 0)

In [98]:
# using Label Powerset
from skmultilearn.problem_transform import LabelPowerset
from sklearn.naive_bayes import GaussianNB

# initialize Label Powerset multi-label classifier
# with a gaussian naive bayes base classifier
classifier = LabelPowerset(GaussianNB())

# train
classifier.fit(samxtrain, samytrain)

# predict
predictions = classifier.predict(samxtest)
from sklearn.metrics import roc_auc_score
roc_auc_score(samytest,predictions.toarray(),average='samples')

0.5554965551518193

In [109]:
# using Label Powerset
from skmultilearn.problem_transform import LabelPowerset
from sklearn.naive_bayes import GaussianNB

# initialize Label Powerset multi-label classifier
# with a gaussian naive bayes base classifier
classifier = LabelPowerset(GaussianNB())

# train
classifier.fit(samxtrain, samytrain)

# predict
predictions = classifier.predict(samxtrain)
from sklearn.metrics import roc_auc_score
roc_auc_score(samytrain,predictions.toarray(),average='samples')

0.6824250275661522

In [112]:
from skmultilearn.adapt import MLkNN

classifier = MLkNN(k=10,s=0.5)

# train
classifier.fit(X_train, Y_train)

# predict
predictions = classifier.predict(X_test)
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_test.toarray(),predictions.toarray(),average='samples')

0.5098644325237893

In [111]:
from skmultilearn.adapt import MLkNN

classifier = MLkNN(k=10,s=0.5)

# train
classifier.fit(X_train, Y_train)

# predict
predictions = classifier.predict(X_train)
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_train.toarray(),predictions.toarray(),average='samples')

0.5187412047203723

<h1><u>PERFORMANCE</u></h1> 

The approaches used in the above multi-label classification are:

<h2><b>Problem Transformation:</b></h2> In thos method we try to transform our multi-label problem into single-label problems. One of the ways to achieve this is by using label powerset with Naive Bayes classification algorithm. <b>The train data accuracy is 68.2% and the test data accuracy is 55.5%, which is higher than the accuracy specified in the report provided.</b>

<h2><b>Adapted Algorithm:</b></h2> This method suggests adapting the algorithm to directly perform multi-label classification, rather than transforming the problem into different subsets of problems.One of the ways to implement this is ny using multi-label version of kNN (MLkNN). the train data accuarcy is 51.8% and test data accuracy is 50.9%

Ref:https://www.analyticsvidhya.com/blog/2017/08/introduction-to-multi-label-classification/